This notebook is based on Chapter 9 (?) of Bishop, and aims to replicate the 'variational mixture of Gaussians' example worked through there. This is to help my understanding of this example, as much of it is the basis for the gradient ascent approach used in my 4YP, and to hopefully get functioning posterior inference. I'm stuck in a rut, and better to go from a working model than try and debug a crummy bloated one built from scratch that has never worked!

# Intro
Distribution of $Z$:

i.e. the assignment $z_i$ for a single $x_i$ equals $k$ with probability $\pi_k$, where $\sum \limits^K \pi_k = 1$

$p(Z|\pi) = \prod \limits^N \prod \limits^K \pi_{k}^{z_{nk}}$

Conditional distribution of observed data (i.e. dataset given true labels and Gaussian means/precisions):

$p(X|Z,\mu,\Lambda) = \prod \limits^N \prod \limits^K \mathcal{N}(x_n | \mu_k, \Lambda_k^{-1})^{z_{nk}}$




#### The joint distribution

Factorises as follows:

$p(X,Z,\pi,\mu,\Lambda) = p(X|Z,\mu,\Lambda)p(Z|\pi)p(\pi)p(\mu|\Lambda)p(\Lambda)$


#### The variational distribution

Takes the following form - the only assumption is that it factorises between latent variablesd $Z$ and the parameters:

$q(Z,\pi,\mu,\Lambda) = q(Z)q(\pi,\mu,\Lambda)$

The fundamental general expression for the iterative variaitional approach here is:

$\log q_j^*(Z_j) = \mathbb{E}_{i \neq j}[\log p(X,Z)] + c$

i.e. look at the expectation of the joint of $\log p$ for all the latent variables except the one we are predicting - in our case for considering just $Z$ this is:

$\log q^*(Z) = \mathbb{E}_{\pi,\mu,\Lambda}[\log p(X,Z,\pi,\mu,\Lambda)] + c$

(N.b. when we write $\mathbb{E}_{\pi,\mu,\Lambda}$, what this really is is $\mathbb{E}_{q(\pi,\mu,\Lambda)}$)

And then decompose the joint into $\mathbb{E}_\pi[\log p(Z|\pi)] + \mathbb{E}_{\mu,\Lambda}[\log p(X|Z,\mu,\Lambda)] + c$

Some reworking and taking the exponent again brings us to:

$q^*(Z) = \prod \limits^N \prod \limits^K r_{nk}^{z_{nk}}$
where $r_{nk} = \dfrac{\rho_{nk}}{\sum \limits^K_{j=1} \rho_{nj}}$ and $\rho_{nk} = \pi_k \mathcal{N}(x_n|\mu,\Lambda_k^{-1})$

i.e. we have derived that the form of the $Z$ factor of the variaitonal distribution takes the form of a multinomial, with a responsibility of each mixture component $k$ for each datapoint $n$


This still leaves the other factor of the variational distribution, $q(\pi,\mu,\Lambda)$. Again using the general idea of the expectation over the joint,

$\log q^*(\pi,\mu,\Lambda) = \mathbb{E}_Z[\log p(X,Z,\pi,\mu,\Lambda)] =\\ \mathbb{E}_Z[\log p(X|Z,\mu,\Lambda) + \log p(Z|\pi) + \log p(\pi) + \log p(\mu,\Lambda)]$

The last two terms simply equal themselves as they don't have a $Z$ component (can go outside of integral and the expectiation is simply an integral over a distribution summing to 1), but the first two have an expectation.

From choice of priors, $p(\pi)$ is a Dirichlet distribution $= C(\alpha_0)\prod \limits^K \pi_k^{\alpha_0-1}$

$p(\mu,\Lambda)$ is a factorised product of Gaussian-Wishart: $p(\mu|\Lambda)p(\Lambda) = \prod \limits^K \mathcal{N}(\mu_k|m_0,(\beta_0 \Lambda)^{-1})\mathcal{W}(\Lambda_k|W_0,\nu_0)$

#### Gaussian-Wishart parameters
https://math.stackexchange.com/questions/2803164/degrees-of-freedom-in-a-wishart-distribution

$m_0$ is the 'location' of the mean $\mu$ chosen usually as zero by symmetry

$\beta_0$: choosing a large $\beta_0$ tightens the Gaussian from which $\mu$ is sampled.

$W_0$ is the positive definite scale matrix

If we have some prior idea of the covariance matrix $\Lambda^{-1}$, $\Sigma_0$, we use $\nu$ and $W$ to encode our confidence in it. We set $W = \Sigma_0^{-1}$. The mean (expectation) of a Wishart is $\nu W$ and its mode is $(\nu-d-1) W$. the least informative prior is $\nu=d$, or possibly $d+2$ - something small. Choosing a large $\nu$ encodes confidence in the prior $\Sigma_0$. 

back to the variational distribution. There are clearly two parts, $q(\pi)$ and $q(\mu,\Lambda)$.

$q(\pi) = \log p(\pi) + \mathbb{E}_Z[\log p(Z|\pi)] = \log C(\alpha_0) + \log(\alpha_0 - 1) \sum \limits^K \pi_k + \sum \limits^{N,K} \mathbb{E}[z_{nk}] \log \pi_k$

$\mathbb{E}_Z[z_{nk}]$ is the expectation of a multinomial, i.e. number of trials*probability, $=r_{nk}$

$q^*(\pi) = C_1 \prod \limits^K \pi_k^{\alpha_0-1}\prod \limits^N \pi_k^{r_{nk}}= C_1\prod \limits^K \pi_k^{(\alpha_0-1+\sum \limits^N r_{nk})} = Dir(\pi|\alpha)$ where $\alpha_k = \alpha_0 + \sum \limits^N r_{nk}$

Cool!

And now for $\log q^*(\mu,\Lambda)$:

$\log q^*(\mu,\Lambda) = \sum \limits^K \log p(\mu_k,\Lambda_k) + \mathbb{E}_Z[\log p(X|Z,\mu,\Lambda)] = \\
\sum \limits^K \log (\mathcal{N}(\mu_k|m_0,\beta_0,\Lambda_k) \mathcal{W}(\Lambda_k|\nu,W_0)) + \sum \limits^{N,K} \mathbb{E}_Z[z_{nk}]\log \mathcal{N}(x_n|\mu_k,\Lambda_k^{-1})$

Similar to q*(\pi) this turns into a form of a Gaussian-Wishart with different parameters.

In [ ]:
!pip install autograd
import autograd
import autograd.numpy as np
from autograd.numpy.linalg import inv, det, multi_dot
from autograd.scipy.special import digamma, gammaln
import os
import matplotlib.pyplot as plt
from utils import *

In [ ]:
def alpha_k(Nk, alpha0):
  return alpha0 + Nk

def beta_k(Nk, beta0):
  return beta0 + Nk

def m_k(Nk, xkbar, betak, m0, beta0):
  return ((1/betak)*(beta0*m0 + Nk*xkbar)).reshape(2)

def W_k(Nk, xkbar, Sk, m0, beta0):
  inv_Wk = inv(W0) + Nk*Sk + ((beta0*Nk)/(beta0+Nk))*np.dot((xkbar-m0),(xkbar-m0).T) 
  return inv(inv_Wk)

def nu_k(Nk, nu0):
  return nu0 + Nk

These use some new parameters, $N_k, \bar{x}_k, S_k$, which essentially correspond to the sum of responsibilities, responsibility-weighted mean and responsibility-weighted variance for mixture component $k$:

In [ ]:
def N_k(responsibilities_k):
  return np.sum(np.array(responsibilities_k))

def x_k_bar(Nk, responsibilities_k, X):
  sum = np.zeros((1,D))
  for n in range(N):
    sum = sum + responsibilities_k[n]*X[n]
  if Nk > 0:
    return (1/Nk)*sum
  else:
    return 0.

def S_k(Nk, responsibilities_k, X, xkbar):
  sum = 0.0
  for n in range(N):
    sum = sum + responsibilities_k[n]*np.dot((X[n]-xkbar).T,(X[n]-xkbar))
  if Nk > 0:
    return (1/Nk)*sum
  else:
    return np.eye(2) # doesn't actually matter what it returns, the component is dead

These updates correspond to the M (maximisation) step of the EM algorithm. To perform them we of course need our responsibilities:

$\mathbb{E}_Z[r_{nk}] = \dfrac{\rho_{nk}}{\sum \limits^K_{j=1} \rho_{nj}}$ and
$\rho_{nk} = \pi_k \mathcal{N}(x_n|\mu,\Lambda_k^{-1})$

The $\rho_{nk}$ above is not quite right --- we have ignored the $\mathbb{E}_{\mu,\lambda}[.]$ Let's consider in full:

$\begin{aligned}
\ln \rho_{n k}=& \mathbb{E}\left[\ln \pi_{k}\right]+\frac{1}{2} \mathbb{E}\left[\ln \left|\boldsymbol{\Lambda}_{k}\right|\right]-\frac{D}{2} \ln (2 \pi) \\
&-\frac{1}{2} \mathbb{E}_{\mu_{k}, \Lambda_{k}}\left[\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)^{\mathrm{T}} \boldsymbol{\Lambda}_{k}\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)\right]
\end{aligned}$

There are four components here:


1.   $\mathbb{E}[\ln \pi_{k}] = \psi(\alpha_k) - \psi(\sum \limits^K \alpha_k)$, using the digamma function $\psi(x) = \dfrac{d}{dx}\log \Gamma = \dfrac{\Gamma '}{\Gamma}$. This is the known log-expectation of the Dirichlet distribution.
2.   $ \mathbb{E}\left[\ln \left|\boldsymbol{\Lambda}_{k}\right|\right] = \sum_{i=1}^{D} \psi\left(\frac{\nu_{k}+1-i}{2}\right)+D \ln 2+\ln \left|\mathbf{W}_{k}\right|$. This is the known log-expectation of the Wishart distribution.
3. Easy
4. $\mathbb{E}_{\boldsymbol{\mu}_{k}, \boldsymbol{\Lambda}_{k}}\left[\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)^{\mathrm{T}} \boldsymbol{\Lambda}_{k}\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)\right] = D\beta_{k}^{-1}+\nu_{k}\left(\mathbf{x}_{n}-\mathbf{m}_{k}\right)^{\mathrm{T}} \mathbf{W}_{k}\left(\mathbf{x}_{n}-\mathbf{m}_{k}\right)$. The proof of this is fiddly but it does work (see https://github.com/zhengqigao/PRML-Solution-Manual, p209).


In [ ]:
def E_ln_pi_k(k, alpha):
  return digamma(alpha[k]) - digamma(np.sum(alpha))

def E_ln_lam_k(k, nu, W):
  return np.sum(digamma(nu[k]+1-np.arange(D)+1)) + D*np.log(2) + np.log(det(W[k]))

def E_ln_mu_k(k, beta, m, W, nu, xn):
  # print('m, W\n', m, W)
  return D*beta[k]**-1 + nu[k]*multi_dot(((xn-m[k]), W[k], (xn-m[k]).T))

def ln_rho_nk(k, alpha, nu, W, beta, m, xn):
  return E_ln_pi_k(k,alpha) + 0.5*E_ln_lam_k(k, nu, W) - 0.5*D*np.log(2*np.pi) - 0.5*E_ln_mu_k(k, beta, m, W, nu, xn)

Now we need $r_{nk}$, which we remember is $\rho_{nk}$ divided by the sum of $\rho_{nk}$ over $k$:

In [ ]:
def r_nk(k, alpha, nu, W, beta, m, xn):
  rhonk = np.exp(ln_rho_nk(k, alpha, nu, W, beta, m, xn))
  sum_k_rho = 0.
  for j in range(K):
    sum_k_rho = sum_k_rho + np.exp(ln_rho_nk(j, alpha, nu, W, beta, m, xn))
  return rhonk/sum_k_rho

### The algorithm
E step: Calculate responsibilities $r_{nk}$ for each and every mixture component $k$ and datapoint $x_n$.

M step: use $r_{nk}$ in the update equations for $\alpha, \beta, m, W, \nu$ to minimise the $\mathbb{KL}$ divergence between the variational distribution $q(Z,\pi,\mu,\Lambda)$ and the true posterior $p(Z,\pi,\mu,\Lambda|X)$

In [ ]:
# E step: calculate responsibility
def E_step(N,K,alpha,nu,W,beta,m,X):
  r = []
  for k in range(K):
    r.append([])
    for n in range(N):
      r[k].append(r_nk(k, alpha, nu, W, beta, m, X[n]))
  return r

In [ ]:
# M step: update hyperparameters
def M_step(r,X,alpha0,beta0,m0,W0,nu0):
  NK, xbar, S = [],[],[]
  alpha, beta, nu = np.empty(K), np.empty(K), np.empty(K)
  m, W = [np.zeros(2) for _ in range(K)], [np.zeros((2,2)) for _ in range(K)]

  for k in range(K):
    Nk = N_k(r[k])
    xkbar = x_k_bar(Nk, r[k], X)
    Sk = S_k(Nk, r[k], X, xkbar)

    alpha[k] = alpha_k(Nk, alpha0)
    beta[k] = beta_k(Nk, beta0)
    m[k] = m_k(Nk, xkbar, beta[k], m0, beta0)
    W[k] = W_k(Nk, xkbar, Sk, m0, beta0)
    nu[k] = nu_k(Nk, nu0)

    NK.append(Nk)
    xbar.append(xkbar)
    S.append(Sk)

    # print('k=%d\nalpha'%k, alpha[k], '\nbeta', beta[k], '\nm', m[k], '\nW', W[k], '\nnu', nu[k])
  return alpha, beta, m, W, nu, NK, xbar, S

## Calculating ELBO
Now lets get the programme to calculate the evidence lower bound on each iteration. From Bishop, eq.10.70:

\begin{aligned}
\mathcal{L}=& \sum_{\mathbb{Z}} \iiint q(\mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda}) \ln \left\{\frac{p(\mathbf{X}, \mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})}{q(\mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})}\right\} \mathrm{d} \boldsymbol{\pi} \mathrm{d} \boldsymbol{\mu} \mathrm{d} \boldsymbol{\Lambda} \\
=& \mathbb{E}[\ln p(\mathbf{X}, \mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})]-\mathbb{E}[\ln q(\mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})] \\
=& \mathbb{E}[\ln p(\mathbf{X} \mid \mathbf{Z}, \boldsymbol{\mu}, \boldsymbol{\Lambda})]+\mathbb{E}[\ln p(\mathbf{Z} \mid \boldsymbol{\pi})]+\mathbb{E}[\ln p(\boldsymbol{\pi})]+\mathbb{E}[\ln p(\boldsymbol{\mu}, \boldsymbol{\Lambda})] \\
&-\mathbb{E}[\ln q(\mathbf{Z})]-\mathbb{E}[\ln q(\boldsymbol{\pi})]-\mathbb{E}[\ln q(\boldsymbol{\mu}, \boldsymbol{\Lambda})]
\end{aligned}

In [ ]:
# components of ELBO

def E_ln_p_X_given_Z_mu_lam(beta, m, W, nu, NK, S, xbar):
  sum = 0.
  for k in range(K):
    Eln_lam = E_ln_lam_k(k, nu, W)
    Tk = nu[k]*np.trace(np.dot(S[k],W[k]))
    Eln_mu = E_ln_mu_k(k,beta,m,W,nu,xbar[k])
    # print('\n%d\nEln_lam: '%k, Eln_lam)
    # print('Tl: ', Tk)
    # print('Eln_mu: ', Eln_mu)
    # print('NK: ', NK)
    sum = sum + NK[k]*(Eln_lam - Eln_mu - Tk - D*np.log(2*np.pi))
  #print(sum)
  #print(sum[0][0])
  return 0.5*sum

def E_ln_p_Z_given_pi(r, alpha):
  sum = 0.
  for n in range(N):
    for k in range(K):
      sum = sum + r[k][n]*E_ln_pi_k(k,alpha)
  return sum

def ln_C(alpha):
  return gammaln(np.sum(alpha)) - np.sum(gammaln(alpha))

def ln_B(W,nu):
  ln_num = -0.5*nu*np.log(det(W))
  ln_det_1 = 0.5*nu*D*np.log(2) - D*(D-1)*0.25*np.log(np.pi)
  ln_det_2 = np.sum(gammaln(np.array([0.5*(nu+1-i) for i in range(D)])))
  return  ln_num - ln_det_1 - ln_det_2

def E_ln_p_pi(alpha0, alpha):
  return ln_C(alpha0*np.ones(K)) + (alpha0-1)*np.sum(np.array([E_ln_pi_k(k,alpha) for k in range(K)]))

def E_ln_p_mu_lam(beta, m, W, nu, m0, W0, nu0):
  sum1, sum2, sum3 = 0., 0., 0.
  for k in range(K):
    F = beta0*E_ln_mu_k(k, beta, m, W, nu, m0)
    Eln_lam = E_ln_lam_k(k, nu, W)
    sum1 = sum1 + D*np.log(beta0/(2*np.pi)) + Eln_lam - F

    sum2 = sum2 + 0.5*(nu0-D-1)*E_ln_lam_k(k,nu,W)
    sum3 = sum3 + nu[k]*np.trace(np.dot(inv(W0),W[k]))
  lnB = ln_B(W0,nu0)
  return 0.5*sum1 + sum2 - 0.5*sum3 + K*lnB

def E_ln_q_Z(r):
  sum = 0.
  for n in range(N):
    for k in range(K):
      if r[k][n] > 0: # n.b. returns nan for 0log0, so need to bypass
        sum = sum + r[k][n]*np.log(r[k][n])
  return sum

def E_ln_q_pi(alpha):
  return np.sum(np.array([(alpha[k]-1)*E_ln_pi_k(k,alpha) for k in range(K)])) + ln_C(alpha)

def H_q_lam_k(k,nu,W):
  return -ln_B(W[k],nu[k]) - 0.5*(nu[k]-D-1)*E_ln_lam_k(k, nu, W) + 0.5*nu[k]*D

def E_ln_q_mu_lam(beta,W,nu):
  sum = 0.
  for k in range(K):
    sum=sum+0.5*E_ln_lam_k(k,nu,W)+0.5*D*np.log(beta[k]/(2*np.pi))-0.5*D-H_q_lam_k(k,nu,W)
  return sum

def calculate_ELBO(r,alpha,beta,m,W,nu,NK,S,xbar,alpha0,m0,W0,nu0):
  p1 = E_ln_p_X_given_Z_mu_lam(beta, m, W, nu, NK, S, xbar)
  p2 = E_ln_p_Z_given_pi(r, alpha)
  p3 = E_ln_p_pi(alpha0, alpha)
  p4 = E_ln_p_mu_lam(beta, m, W, nu, m0, W0, nu0) # this one is an array
  q1 = E_ln_q_Z(r)
  q2 = E_ln_q_pi(alpha)
  q3 = E_ln_q_mu_lam(beta,W,nu)
  # print(type(p1),type(p2),type(p3),type(p4),type(q1),type(q2),type(q3))
  # print(p1)
  # if np.sum(np.isnan([p1,p2,p3,p4,q1,q2,q3]))>0:
  #   print('\nArgs of calculate ELBO:\nr,alpha,beta,m,W,nu,NK,S,xbar,alpha0,m0,W0,nu0')
  #   print(r,alpha,beta,m,W,nu,NK,S,xbar,alpha0,m0,W0,nu0)
  # assert not np.isnan(p1)
  # assert not np.isnan(p2)
  # assert not np.isnan(p3)
  # assert not np.isnan(p4)
  # assert not np.isnan(q1)
  # assert not np.isnan(q2)
  # assert not np.isnan(q3)
    
  return p1+p2+p3+p4-q1-q2-q3



# Code


1.   Choose some priors, $\mathbf{\theta}_0$, set $\mathbf{\theta} = \mathbf{\theta}_0$ 
2.   Iterate until converged:   
  1.   Calculate the full set of $NK$ responsibilities
  2.   Using update equations, update each $\theta_k$

In [ ]:
from numpy.random import multivariate_normal
import matplotlib.pyplot as plt
if not os.path.exists('plots'):
  os.mkdir('plots')
if not os.path.exists('gifs'):
  os.mkdir('gifs')

# Set priors, dataset, problem
D = 2       # dimensions
K = 3       # number of components, unnecessary ones should go to zero
N = 500     # number of points in synthetic dataset
N_its = 100 # number of updates

# Dataset
centres = [np.array([0.,8.]), np.array([5.,0.])]
covs = [np.eye(2), np.array([[0.6,0.4],
                             [0.4,0.6]])] 
X1 = multivariate_normal(mean=centres[0],
                         cov=covs[0],
                         size=int(N/2))
X2 = multivariate_normal(mean=centres[1],
                         cov=covs[1],
                         size=int(N/2))
X = np.concatenate((X1,X2))

In [ ]:
# Variational priors
alpha0 = 1e-3     # as alpha0 -> 0, pi_k -> 0. As alpha0 -> Inf, pi_k -> 1/K
beta0 = 1e-10     # ???
m0 = np.zeros(2)  # zero by convention (symmetry)
W0 = np.eye(2)    # 
nu0 = 2           # 

In [ ]:
%matplotlib notebook
# hacky, but this suppresses plots

# r needs to be randomised (not uniform) because if its all the same nothing 
# changes - not sure of the mathematical reasoning for this
r = np.array([np.random.dirichlet(np.ones(K)) for _ in range(N)])
r = [r[:,k] for k in range(K)]

# for plotting
# alphas, betas, Ws, ms, nus = [],[],[],[],[]
alphas, betas = np.zeros((N_its,K)), np.zeros((N_its,K))
ms = np.zeros((N_its,K,D))
varx, vary, covxy = np.empty((N_its,K)),np.empty((N_its,K)),np.empty((N_its,K)) 

# neaten up the output
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

verbose = False

ELBO, ELBO_M, ELBO_E = np.empty(2*N_its), np.empty(N_its), np.empty(N_its)
for i in tqdm(range(N_its)):
  # M step
  alpha, beta, m, W, nu, NK, xbar, S = M_step(r,X,alpha0,beta0,m0,W0,nu0)
  # alphas[i],betas[i],ms[i],Ws[i],nus[i] = alpha,beta,m,W,nu
  alphas[i,:] = alpha
  betas[i,:] = beta
  # m is shape K,D when cast to nparray
  ms[i,:,:] = np.array(m) # shape N_its*K*D
  varx[i,:] = np.array([inv(S[k])[0,0] for k in range(K)]) 
  vary[i,:] = np.array([inv(S[k])[1,1] for k in range(K)]) 
  covxy[i,:] = np.array([inv(S[k])[1,0] for k in range(K)]) 

  def E_pi(alpha, alpha0, N):
    return [(alpha[k])/(K*alpha0 + N) for k in range(K)]
  
  Epi = E_pi(alpha, alpha0, N)
  ELBO[2*i] = calculate_ELBO(r,alpha,beta,m,W,nu,NK,S,xbar,alpha0,m0,W0,nu0)
  ELBO_M[i] = ELBO[2*i]
  
  if verbose:
    print('\n******************Iteration %d************************\n'%i)
    print('\nalpha', alpha, '\nbeta', beta, '\nnu', nu, '\nm', m, '\nW', W, '\nnu', nu)
    print('E[pi] = ', Epi)
    print('ELBO = %f'%ELBO[i])

  # Plot
  title = 'iteration %d' % i
  filename = 'plots/img%04d.png'%i
  # plot_GMM(X, mu, lam, pi, centres, covs, K, title)
  plot_GMM(X, m, inv(S), Epi, centres, covs, K, title, savefigpath=filename)

  # E step
  r = E_step(N,K,alpha,nu,W,beta,m,X)
  ELBO[2*i+1] = calculate_ELBO(r,alpha,beta,m,W,nu,NK,S,xbar,alpha0,m0,W0,nu0)
  ELBO_E[i] = ELBO[2*i+1]

In [ ]:
# Make and display gif 
filedir = 'plots'
gifdir = 'gifs'
gifname = make_gif(filedir, gifdir)

# delete pngs for next run
for file in os.listdir(filedir):
  os.remove(os.path.join(filedir,file))

Image(open(gifdir+"/%s.gif"%gifname,'rb').read())

##Plot everything else
Lets see how alpha, beta, m, W, nu change over iterations 

In [ ]:
%matplotlib inline
fig=plt.figure(figsize=(14,6), dpi= 100, facecolor='w', edgecolor='k')
plt.plot(np.arange(0,N_its,0.5), ELBO)
plt.plot(ELBO_M)
plt.plot(np.arange(N_its)+0.5, ELBO_E)
plt.legend(['overall', 'after M step', 'after E step'])
plt.xlabel('Iterations')
plt.ylabel('Evidence lower bound')
plt.show();

In [ ]:
%matplotlib inline
legends = []
for k in range(K):
  plt.plot(alphas[:,k].T)
  legends.append('k=%d'%k)
plt.legend(legends)
plt.title('alphas')
plt.xlabel('Iterations')
plt.ylabel('alpha')

In [ ]:
%matplotlib inline
legends = []
for k in range(K):
  plt.plot(betas[:,k].T)
  legends.append('k=%d'%k)
plt.legend(legends)
plt.title('betas')
plt.xlabel('Iterations')
plt.ylabel('beta')

In [ ]:
%matplotlib inline
fig=plt.figure(figsize=(14,10), dpi= 100, facecolor='w', edgecolor='k')
legends = []
for k in range(K):
  # ms is N_its*K*D 
  plt.plot(ms[:,k,0], ms[:,k,1])
  legends.append('k=%d'%k)
  startx, starty, endx, endy = ms[0,k,0],ms[0,k,1],ms[-1,k,0],ms[-1,k,1]
  plt.text(startx, starty, '0')
  plt.text(endx, endy, '%d'%N_its)
plt.legend(legends)
plt.title('m')
plt.plot(np.array([centres[0][0],centres[1][0]]),
             np.array([centres[0][1],centres[1][1]]),
             'rx')


In [ ]:
%matplotlib inline
legends = []
for k in range(K):
  plt.figure()
  plt.plot(varx[:,k].T, 'r')
  plt.plot(vary[:,k].T, 'b')
  plt.plot(covxy[:,k].T, 'g')
  legends += ['var_x', 'var_y', 'cov_xy']
  plt.legend(legends)
  plt.title('Covariance matrix k=%d'%k)
  plt.xlabel('Iterations')

## Gradient ascent optimisation of ELBO (MOVED TO SPYDER)

From Bishop:

*Finally, it is worth noting that the lower bound provides an alternative approach for deriving the variational re-estimation equations ... To do
this we use the fact that, since the model has conjugate priors, the functional form of the factors in the variational posterior distribution is known, namely discrete for $Z$, Dirichlet for $\pi$, and Gaussian-Wishart for $(\mu_k,\lambda_k)$. By taking general parametric forms for these distributions we can derive the form of the lower bound as a function of the parameters of the distributions. Maximizing the bound with respect to these parameters then gives the required re-estimation equations.*

i.e. ELBO is a function of the distributional parameters $\mathbf{\theta} = \alpha,\beta,m,W,\nu$. Therefore we can calculate $\nabla_\theta(ELBO)$ and perform gradient ascent to maximise it and find the variational parameters of $q$, rather than using iterative update equations.

In [ ]:

def ELBO_theta(alpha,beta,m,W,nu,X,alpha0,m0,W0,nu0):
  r, NK  = [],[]
  xbar, S = [],[] 
  for k in range(K):
    r.append([])
    for n in range(N):
      r[k].append(r_nk(k, alpha, nu, W, beta, m, X[n]))
  for k in range(K):
    NK.append(N_k(r[k]))
    xbar.append(x_k_bar(NK[k], r[k], X))
    S.append(S_k(NK[k], r[k], X, xbar[k]))
  return calculate_ELBO(r, alpha, beta, m, W, nu, NK, S, xbar, alpha0, m0, W0, nu0)

# ELBO_theta(alpha,beta,m,W,nu,X,alpha0,m0,W0,nu0)


The problem at the moment is that autograd does not support indexing, e.g. `r[n,k] = XXX`. I have instead swapped to `r[k][n]`, i.e. a list of K lists of length N, building it by `.append` rather than assignment.

In [ ]:
%%capture
# setup stuff
from autograd import grad
D_alpha = grad(ELBO_theta, 0)
D_beta = grad(ELBO_theta, 1)
D_m = grad(ELBO_theta, 2)
# D_W = grad(ELBO_theta, 3)

r = np.array([np.random.dirichlet(np.ones(K)) for _ in range(N)])
r = [r[:,k] for k in range(K)]
alpha, beta, m, W, nu, NK, xbar, S = M_step(r,X,alpha0,beta0,m0,W0,nu0)

N_its = 10
step_alpha, step_beta, step_m, step_W = 1.0, 1.0, .1, .1
ELBO = np.empty(N_its+1)
ELBO[0] = ELBO_theta(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)


In [ ]:
# Iterate
for i in range(N_its):
  print('\n*****************')

  d_alpha = D_alpha(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
  d_beta = D_beta(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
  d_m = D_m(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)  
  # d_W = D_W(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
  
  try: d_alpha = d_alpha._value
  except: pass
  try: d_beta = d_beta._value
  except: pass
  try: d_m = d_m._value
  except: print('not an arraybox i guess: ', type(d_m))
  # try: d_W = d_W._value
  # except: print('not an arraybox i guess: ', type(d_W))

  alpha = alpha + step_alpha*d_alpha
  beta = beta + step_beta*d_beta
  m = [m[k] + step_m*d_m[k] for k in range(K)]
  # W = [W[k] + step_W*d_W[k] for k in range(K)]
  print('\nalpha: ', alpha)
  print('\nbeta: ', beta)
  print('\nm: ', m)
  # print('\nW: ', W)

  ELBO[i+1] = ELBO_theta(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
  print('\n%d: '%i, ELBO[i+1]-ELBO[i], (ELBO[i+1]-ELBO[i])>0)

In [ ]:
ELBO

In [ ]:
ELBO.shape

In [ ]:
# messing around with numerical differentiation

def D_alpha(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0):
  d = 1e-2
  alphapl = alpha+d
  alphami = alpha-d
  Lpl = ELBO_theta(alphapl, beta, m, W, nu, X, alpha0, m0, W0, nu0)
  Lmi = ELBO_theta(alphami, beta, m, W, nu, X, alpha0, m0, W0, nu0)
  return (Lpl-Lmi)/(2*d)

def D_m(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0):
  d = 1e-2
  m_pl = [m[k]+d for k in range(K)]
  m_mi = [m[k]-d for k in range(K)]
  Lpl = ELBO_theta(alpha, beta, m_pl, W, nu, X, alpha0, m0, W0, nu0)
  Lmi = ELBO_theta(alpha, beta, m_mi, W, nu, X, alpha0, m0, W0, nu0)
  return (Lpl-Lmi)/(2*d)

In [ ]:
# it doesnt work because you need the gradient for each K component. in fact since m is D*K
# so really you need to work out grad for a lot of fucking paremeters - hate to say it but
# autograd is going to be a necessity

# r = np.array([np.random.dirichlet(np.ones(K)) for _ in range(N)])
# alpha, beta, m, W, nu, NK, xbar, S = M_step(r,X,alpha0,beta0,m0,W0,nu0)

# for i in range(100):
#   old = ELBO_theta(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
#   d_alpha = D_alpha(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
#   d_m = D_m(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
#   print('d_m', d_m)
#   alpha = alpha + d_alpha
#   m = m + d_m
#   new = ELBO_theta(alpha, beta, m, W, nu, X, alpha0, m0, W0, nu0)
#   print('%d: '%i, new-old, (new-old)>0)